# Modeling Your Data - Lab

## Introduction 

In this lab you'll perform a full linear regression analysis on the data. You'll implement the process demonstrated in the previous lesson, taking a stepwise approach to analyze and improve the model along the way.

## Objectives
You will be able to:

* Perform a full linear regression with iterations based on p-value of features and other parameters
* Create visualizations to better understand the distributions of variables in a dataset
* Determine whether or not the assumptions for linear regression hold true for this example

## Build an Initial Regression Model

To start, perform a train-test split and create an initial regression model to model the `list_price` using all of your available features.

> **Note:** In order to write the model you'll have to do some tedious manipulation of your column names. Statsmodels will not allow you to have spaces, apostrophe or arithmetic symbols (+) in your column names. Preview them and refine them as you go.  
**If you receive an error such as "PatsyError: error tokenizing input (maybe an unclosed string?)", then you need to further preprocess your column names.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

In [4]:
# Import the dataset 'Lego_dataset_cleaned.csv'
df = pd.read_csv('Lego_dataset_cleaned2.csv')
df.head()

,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10+,ages_10-14,ages_10-16,ages_10-21,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very Challenging,review_difficulty_Very Easy,review_difficulty_unknown
0,-0.273020,29.99,-0.398512,-0.655279,-0.045687,-0.365010,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,-0.404154,19.99,-0.398512,-0.655279,0.990651,-0.365010,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,-0.517242,12.99,-0.147162,-0.132473,-0.460222,-0.204063,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,0.635296,99.99,0.187972,-1.352353,0.161581,0.117830,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0.288812,79.99,-0.063378,-2.049427,0.161581,-0.204063,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [8]:
# Your code here - Manipulate column names
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_')
       ]
def col_formatting(col):
    for old, new in subs:
        col = col.replace(old,new)
    return col

In [11]:
df.columns = [col_formatting(col) for col in df.columns]
df.head()

,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10plus,ages_10_14,ages_10_16,ages_10_21,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very_Challenging,review_difficulty_Very_Easy,review_difficulty_unknown
0,-0.273020,29.99,-0.398512,-0.655279,-0.045687,-0.365010,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,-0.404154,19.99,-0.398512,-0.655279,0.990651,-0.365010,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,-0.517242,12.99,-0.147162,-0.132473,-0.460222,-0.204063,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,0.635296,99.99,0.187972,-1.352353,0.161581,0.117830,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0.288812,79.99,-0.063378,-2.049427,0.161581,-0.204063,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [12]:
# Your code here - Define the target and predictors
outcome = 'list_price'
x_cols = list(df.columns)
x_cols.remove(outcome)

In [14]:
# Your code here - Split into train and test
train, test = train_test_split(df)

print(len(train), len(test))
train.head()

8152 2718


,piece_count,list_price,num_reviews,play_star_rating,star_rating,val_star_rating,ages_10plus,ages_10_14,ages_10_16,ages_10_21,...,country_NZ,country_PL,country_PT,country_US,review_difficulty_Average,review_difficulty_Challenging,review_difficulty_Easy,review_difficulty_Very_Challenging,review_difficulty_Very_Easy,review_difficulty_unknown
7974,-0.156322,48.7878,-0.175090,-0.829547,-1.082024,-1.491638,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10861,0.092713,79.2878,0.104189,0.390333,0.576116,0.117830,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8118,-0.577395,30.4878,-0.286801,-3.792112,-4.812839,-1.652584,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2127,-0.135870,54.8878,-0.342657,1.087407,0.576116,0.922564,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2754,2.154768,155.9922,0.997879,0.564601,0.783383,0.922564,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [15]:
# Your code here - Fit the actual model
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             list_price   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     537.2
Date:                Wed, 11 Nov 2020   Prob (F-statistic):               0.00
Time:                        19:26:23   Log-Likelihood:                -40446.
No. Observations:                8152   AIC:                         8.108e+04
Df Residuals:                    8057   BIC:                         8.175e+04
Df Model:                          94                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                           65.0880      1.779     36.579      0.000      61.600      68.576
piece_count                                         74.3357      0.891     83.443      0.000      72.589      76.082
num_reviews                                          6.1450      0.665      9.244      0.000       4.842       7.448
play_star_rating                                     5.3290      0.629      8.468      0.000       4.095       6.563
star_rating                                         -1.2681      0.702     -1.808      0.071      -2.643       0.107
val_star_rating                                     -8.5087      0.626    -13.593      0.000      -9.736      -7.282
ages_10plus                                        123.7743      6.494     19.059      0.000     111.044     136.504
ages_10_14                                         -22.8013      8.825     -2.584      0.010     -40.100      -5.502
ages_10_16                                         -10.4600      3.923     -2.667      0.008     -18.150      -2.771
ages_10_21                                          47.3909      7.280      6.510      0.000      33.120      61.662
ages_11_16                                         -14.3462      5.835     -2.459      0.014     -25.784      -2.909
ages_12plus                                         69.3753      5.341     12.988      0.000      58.905      79.846
ages_12_16                                         -55.8551      6.620     -8.437      0.000     -68.832     -42.878
ages_14plus                                         25.4075      4.071      6.240      0.000      17.426      33.389
ages_16plus                                         35.0425      5.944      5.895      0.000      23.390      46.695
ages_1half_3                                        -8.7081      2.369     -3.677      0.000     -13.351      -4.065
ages_1half_5                                        -4.3002      2.928     -1.468      0.142     -10.041       1.440
ages_2_5                                            14.7651      1.653      8.931      0.000      11.524      18.006
ages_4plus                                         -18.1930      6.239     -2.916      0.004     -30.423      -5.963
ages_4_7                                            -5.7776      3.224     -1.792      0.073     -12.097       0.542
ages_4_99                                           -3.0836      3.570     -0.864      0.388     -10.082       3.915
ages_5plus                                          -8.9949      3.437     -2.617      0.009     -15.733      -2.257
ages_5_12                                          -36.1850      3.093    -11.699      0.000     -42.248     -30.122
ages_5_8                                 

## Remove the Uninfluential Features

Based on the initial model, remove those features which do not appear to be statistically relevant and rerun the model.

In [ ]:
# Your code here - Remove features which do not appear to be statistically relevant

In [ ]:
# Your code here - Refit the model

> **Comment:** You should see that the model performance is identical. Additionally, observe that there are further features which have been identified as unimpactful. Continue to refine the model accordingly.

In [ ]:
# Your code here - Continue to refine the model

In [ ]:
# Your code here - Refit the model

## Investigate Multicollinearity

There are still a lot of features in the current model! Chances are there are some strong multicollinearity issues. Begin to investigate the extent of this problem.

In [ ]:
# Your code here - Code a way to identify multicollinearity

## Perform Another Round of Feature Selection

Once again, subset your features based on your findings above. Then rerun the model once again.

In [ ]:
# Your code here - Subset features based on multicollinearity

In [ ]:
# Your code here - Refit model with subset features

## Check the Normality Assumption

Check whether the normality assumption holds for your model.

In [ ]:
# Your code here - Check that the residuals are normally distributed

## Check Homoscedasticity Assumption

Check whether the model's errors are indeed homoscedastic or if they violate this principle and display heteroscedasticity.

In [ ]:
# Your code here - Check that the residuals are homoscedastic

> **Comment:** This displays a fairly pronounced 'funnel' shape: errors appear to increase as the `list_price` increases. This doesn't bode well for our model. Subsetting the data to remove outliers and confining the model to this restricted domain may be necessary. A log transformation or something equivalent may also be appropriate.

## Make Additional Refinements

From here, make additional refinements to your model based on the above analysis. As you progress, continue to go back and check the assumptions for the updated model. Be sure to attempt at least two additional model refinements.

> **Comment:** Based on the above plots, it seems as though outliers are having a substantial impact on the model. As such, removing outliers may be appropriate. Investigating the impact of a log transformation is also worthwhile.

In [ ]:
# Your code here - Check for outliers

In [ ]:
# Your code here
# Remove extreme outliers


# Rerun the model

In [ ]:
# Your code here - Check normality assumption

In [ ]:
# Your code here - Check the Homoscedasticity Assumption

In [ ]:
# Your code goes here

## Summary

Well done! As you can see, regression can be a challenging task that requires you to make decisions along the way, try alternative approaches, and make ongoing refinements. These choices depend on the context and specific use cases. 